In [1]:
%load_ext autoreload

In [3]:
%autoreload 
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
import plotly.express as px

In [4]:
h2o.init(max_mem_size='10G')

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 16 mins
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_henrikraa_nkrdm3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8.27 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


## Load models

In [12]:
stacked_model = h2o.load_model('/home/henrikraa/models/02H_h2o_ensaemble_preprocessed_means_basic_feng')
xgb_model = h2o.load_model('/home/henrikraa/models/02H_h2o_xgboost_preprocessed_means_basic_feng')

### Predict

In [13]:
X_test = pd.read_csv('../data_feature_enigineering/X_test_selected_preprocessed_mean.csv')
X_test_h2o = h2o.H2OFrame(X_test)
X_test_h2o.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


date_forecast,direct_rad:W,sun_elevation:d,visibility:m,pressure_50m:hPa,relative_humidity_1000hPa:p,day_of_year,air_density_2m:kgm3,cloud_base_agl:m,ceiling_height_agl:m,wind_speed_u_10m:ms,sun_azimuth:d,wind_speed_v_10m:ms,day,wind_speed_10m:ms,total_cloud_cover:p,fresh_snow_24h:cm,fresh_snow_6h:cm,fresh_snow_1h:cm,building
2023-05-01 00:00:00,0,-10.541,29907.5,1006.8,80.275,121,1.28675,1061.55,912.7,2.1,16.0265,3.35,1,3.95,74.95,0,0,0,0
2023-05-01 01:00:00,0,-7.8945,29519.1,1006.3,79.825,121,1.286,1075.1,1482.1,1.925,30.4973,3.3,1,3.825,77.475,0,0,0,0
2023-05-01 02:00:00,0,-3.8155,31009.1,1005.8,78,121,1.28375,1200.4,1791.3,1.75,44.5172,3.2,1,3.65,88.1,0,0,0,0
2023-05-01 03:00:00,2.1,1.4125,34552.5,1005.23,75.625,121,1.282,1179.85,2312.88,1.45,58.083,3.15,1,3.5,68.6,0,0,0,0
2023-05-01 04:00:00,25.45,7.4685,35483.9,1004.75,74.225,121,1.281,920.05,2198.3,1.3,71.341,3.05,1,3.325,66.3,0,0,0,0
2023-05-01 05:00:00,88.225,14.0177,35425.4,1004.35,71.1,121,1.276,1419.3,1957.62,1.4,84.5578,2.875,1,3.2,74.075,0,0,0,0
2023-05-01 06:00:00,181.5,20.7117,33709.8,1003.95,66.6,121,1.26825,1299.4,1903.3,2.275,98.0913,2.675,1,3.55,58.55,0,0,0,0
2023-05-01 07:00:00,266.75,27.1717,26871,1003.62,63.375,121,1.2625,743.3,nan,3.475,112.359,2.35,1,4.2,48.55,0,0,0,0
2023-05-01 08:00:00,315.2,32.9648,20572.6,1003.35,61.525,121,1.25975,805.375,1381.2,3.55,127.79,1.6,1,3.9,61.075,0,0,0,0
2023-05-01 09:00:00,334.375,37.5947,15324.5,1003.2,59.725,121,1.2585,884.95,1429.83,3.35,144.697,0.8,1,3.45,80.125,0,0,0,0


In [15]:
# predict using the two models
preds_stacked = stacked_model.predict(X_test_h2o)
preds_xgb = xgb_model.predict(X_test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [18]:
preds_stacked.shape, preds_xgb.shape

((2160, 1), (2160, 1))

In [16]:
preds_stacked.head()

predict
-2.8245
-4.57066
-2.49529
39.2437
282.006
887.839
1783.21
2778.61
2505.57
2507.64


In [17]:
preds_xgb.head()

predict
3.2186
3.23126
6.36366
4.65412
123.521
785.278
1416.82
2631
2421.82
2393.36


In [19]:
# remove all the negative values in the predictions and replace them with 0
preds_stacked[preds_stacked < 0] = 0

# remove all the negative values in the predictions and replace them with 0
preds_xgb[preds_xgb < 0] = 0

In [22]:
preds_stacked.shape, preds_xgb.shape

((2160, 1), (2160, 1))

In [25]:
# convert the predictions to pandas dataframe
preds_stacked_df = preds_stacked.as_data_frame()
preds_xgb_df = preds_xgb.as_data_frame()

In [28]:
preds_stacked_df.to_csv('../data_results/02H_h2o_ensemble_preprocessed_means_basic_feng.csv', index=True)
preds_xgb_df.to_csv('../data_results/02H_h2o_xgboost_preprocessed_means_basic_feng.csv', index=True)